In [82]:
import random
import string
import numpy as np
 
# Configuration
MAX_NUMBER_OF_MUTATORS = 10
INSERT_MAX_LENGTH = 20
DELETE_MAX_LENGTH = 20

# Mutator signature: def function_name(input_data) -> input_data

def mutator_insert(input_data):
    #print(input_data)
    length = random.randrange(0, INSERT_MAX_LENGTH)
    position = random.randrange(0, len(input_data))
    random_bytes = bytearray()
    #for i in range(0, length):
    #    random_bytes += random.choice(string.printable).encode()
    random_bytes += bytes([random.randrange(0, 255)])
    #print(random_bytes)
    return input_data[:position] + random_bytes + input_data[position:]


def mutator_delete(input_data):
    input_length = len(input_data)
    position = random.randrange(0, input_length)
    count = random.randrange(0, min(input_length, DELETE_MAX_LENGTH))
    return input_data[:position] + input_data[position + count:]
# Mutators to build:
# - delete random parts
# - replace mutator

active_mutators = [
    mutator_insert,
    mutator_delete
]


def mutate_corpus(corpus, run_id):
    random.seed(run_id)
    active_corpus = random.choice(corpus)
    return mutate(active_corpus, run_id)

def mutate(input_data, run_id):
    number_of_mutations = random.randint(0, MAX_NUMBER_OF_MUTATORS)
    for i in range(0, number_of_mutations):
        mutator = random.choice(active_mutators)
        input_data = mutator(input_data)
    return input_data

In [83]:
#base_corpus = [np.random.bytes(10),np.random.bytes(20)]
base_corpus = [b"asdffffhgkhgjhghjg",]
#print(string.printable)

In [84]:
import subprocess

def runner(testcase):
    with open("testcase.bin", "wb") as f:
        f.write(testcase)
        f.close()
    result = subprocess.run(["./mqtt", "testcase.bin"], shell=False, timeout=1, env={"LSAN_OPTIONS" : "detect_leaks=0"})
    return result.returncode

In [85]:
runner(b"asdffffhgkhgjhghjg")

0

In [88]:
#!rm mqtt.c.gcov
#!rm mqtt_pal.gcno
#!rm mqtt.gcno
!gcc -g --coverage -fsanitize=address -Iinclude src/* -o mqtt



# #Dry run to determine base coverage
for c in base_corpus:
    runner(c)

def get_coverage():
    r = subprocess.run(["gcov", "mqtt"], stdout=subprocess.PIPE)
    return int(float(r.stdout.decode('utf-8').split('Lines executed:')[1].split('% of')[0]))
    


run_id = 0

current_coverage = 0
print(f"Base covergae is {current_coverage}%")
while True:
    testcase = mutate_corpus(base_corpus, run_id)
    #print("Testcase - %s", testcase)
    result = runner(testcase)
    coverage =get_coverage()
    
    #check if coverage increased
    if(coverage > current_coverage):
        current_coverage = coverage
        corpus.append(testcase)
        print("Current coverage - " + str(current_coverage) + "%")
        
    if result != 0:
        print(f"Success: {run_id}")
    
    run_id += 1

Base covergae is 0%
Current coverage - 5%
Current coverage - 6%
Current coverage - 8%
Success: 12821
Current coverage - 9%
Success: 31591
Success: 45777
Current coverage - 10%
Success: 108189
Current coverage - 12%
Success: 143353
Success: 170720
Current coverage - 13%
Current coverage - 15%
Current coverage - 16%
Success: 222582
Success: 255420
Current coverage - 17%
Success: 331414
Success: 350010
Success: 387155
Success: 404597
Success: 411650
Success: 419686
Success: 436534
Success: 447942
Success: 451668
Success: 468839
Success: 478740
Success: 520402
Success: 534457
Success: 560802
Success: 567795
Success: 583213
Success: 662539
Success: 681751
Success: 696234
Success: 744614
Success: 773281
Success: 793465
Success: 889731
Success: 924567
Success: 1064636
Success: 1100053
Success: 1107627
Success: 1121605
Success: 1152398
Success: 1164817
Success: 1180228
Success: 1186126
Success: 1199672
Success: 1225045
Success: 1243557
Success: 1246402
Success: 1270409
Success: 1298603
Success

Success: 14606686
Success: 14611211
Success: 14627677
Success: 14757738
Success: 14788367
Success: 14799702
Success: 14814267
Success: 14844319
Success: 14884574
Success: 14889920
Success: 14943135
Success: 14986083
Success: 14990727
Success: 15007249
Success: 15065796
Success: 15119462
Success: 15160170
Success: 15186333
Success: 15197512
Success: 15222565
Success: 15302480
Success: 15307938
Success: 15332958
Success: 15367232
Success: 15414525
Success: 15422444
Success: 15441448
Success: 15448784
Success: 15457910
Success: 15556746
Success: 15565399
Success: 15578705
Success: 15608439
Success: 15662647
Success: 15669509
Success: 15700276
Success: 15806556
Success: 15848671
Success: 15863282
Success: 15886635
Success: 15968446
Success: 16015237
Success: 16016933
Success: 16091650
Success: 16132924
Success: 16134552
Success: 16168704
Success: 16206234
Success: 16298650
Success: 16344645
Success: 16370318
Success: 16386281
Success: 16415639
Success: 16439524
Success: 16559966
Success: 1

KeyboardInterrupt: 

In [87]:
testcase = mutate_corpus(corpus, 12821)
result = runner(testcase)
print(testcase)
print(result)

b'8(=\xda\x06c<\x9e\xd4\xf9T\x82"\xdc\x99]\xce\x07\xef"\x7fV\x8b\xa610\r\x1a\xb0i\xac\xa7*\x822'
0
